# 🍪 [Day 21](https://adventofcode.com/2019/day/21)

In [1]:
def run_program(p, inputs, init_op=0, init_base=0):
    """Intcode as usual"""
    #Inputs are given in reverse order (pop)
    op = init_op
    relative_base = init_base
    last_diagnostic = []
    while p[op] != 99:
        codes = "%05d" % p[op]
        codes = [int(codes[0]), int(codes[1]), int(codes[2]), int(codes[3:])]
        # inputs
        if codes[-1] == 3:
            if not len(inputs):
                # Game is waiting for next move
                break
            assert codes[1] == 0
            p[p[op + 1] + (relative_base if codes[2] == 2 else 0)] = inputs.pop()
            op += 2
        # unary ops
        elif codes[-1] in  [4, 9]:
            # read parameter
            assert codes[1] == 0
            param = p[op + 1]
            if (codes[2] % 2) == 0:
                try:
                    param = p[param + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    param = 0
            # output
            if codes[-1] == 4:
                last_diagnostic.append(param)
            # update relative base
            else:
                relative_base += param
            # next instr
            op += 2
        else:
            # read parameters in correct mode
            x, y = p[op + 1:op + 3]
            if (codes[2] % 2) == 0:
                try:
                    x = p[x + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    x = 0
            if not (codes[1] % 2):
                try:
                    y = p[y + (relative_base if codes[1] == 2 else 0)]
                except IndexError:
                    y = 0
            # Read target and allocate more memory if needed
            target = p[op + 3] + (relative_base if codes[0] == 2 else 0)
            if target >= len(p): 
                p += [0] * (target - len(p) + 1)
            # addition and multiplication
            if codes[-1] in [1, 2]:
                p[target] = x + y if codes[-1] == 1 else x * y
                op += 4
            # Comparison result
            elif codes[-1] == 7:
                p[target] = int(x < y)
                op += 4
            elif codes[-1] == 8:
                p[target] = int(x == y)
                op += 4
            # Jump if eq
            elif (codes[-1] == 5 and x != 0) or (codes[-1] == 6 and x == 0):
                op = y  
            # Jump instruction that failed their test
            else:
                op += 3
    return last_diagnostic, 0, op, relative_base


import numpy as np
def build_map(program):
    """Parse inputs"""
    p = [x for x in program]
    outputs = run_program(p, [])[0]
    grid = [[]]
    for c in outputs:
        if c == 46:
            grid[-1].append(0)
        elif c == 35:
            grid[-1].append(1)
        elif c  == 10:
            grid.append([])
        else:
            print("Initial robot direction:", chr(c))
            grid[-1].append(2)
    return np.array(grid[:-2], dtype=np.int32)


from matplotlib import pyplot as plt
def display_grid(grid, title=None):
    plt.imshow(grid.transpose(), cmap='Accent_r')
    plt.axis('off')
    if title is not None:
        plt.title(title)
    plt.show()


def convolve2d(matrix, kernel):
    """Basic 2d convolutions"""
    lkw = int(np.ceil((kernel.shape[0] - 1) / 2))
    lkh = int(np.ceil((kernel.shape[1] - 1) / 2))
    conv = np.zeros_like(matrix)
    # Pad 
    padded_matrix = np.pad(matrix, 
                           ((lkw, kernel.shape[0] - lkw),
                            (lkh, kernel.shape[1] - lkh)),
                            'constant')
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            conv[i, j] = np.sum(kernel * padded_matrix[i:i+kernel.shape[0], 
                                                       j:j+kernel.shape[1]])
    return conv

def get_alignment(grid):
    """Find intersections with a convolution"""
    flt = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])
    convolution = convolve2d(grid, flt)
    intersects = np.where(convolution == 5)
    alignment = sum([i * j for i, j in zip(*intersects)])
    return alignment


def get_path(grid, display=False):
    """Find the path that goes through all scaffolds"""
    robot = np.where(grid == 2)
    i, j = robot[0][0], robot[1][0]
    path = []
    if display:
        plot = np.zeros_like(grid)
        plot[i, j] = 100
    
    # Distinguish between orientation
    # and move commands
    o = (0, -1)
    def find_nxt_move():
        nonlocal i, j, o, grid
        for new_o in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            if (o[0] * new_o[0]) or (o[1] * new_o[1]): # Can not go back or forth in same direction 
                continue
                
            if ((i + new_o[0] >= 0 and i + new_o[0] < grid.shape[0]) and 
                (j + new_o[1] >= 0 and j + new_o[1] < grid.shape[0]) and
                grid[i + new_o[0], j + new_o[1]] == 1):
                    # Resolve
                    if o[1] == -1:
                        move = 'R' if new_o[0] == 1 else 'L'
                    elif o[1] == 1:
                        move = 'R' if new_o[0] == -1 else 'L'
                    elif o[0] == 1:
                        move = 'R' if new_o[1] == 1 else 'L'
                    elif o[0] == -1:
                        move = 'R' if new_o[1] == -1 else 'L'
                    # Assign new orientation
                    o = new_o
                    return move
        o = None
        
    # Find the path in a greedy manner
    forward = 0
    while o != None:
        # Go forward until falling down
        while ((i >= 0 and i < grid.shape[0]) and 
               (j >= 0 and j < grid.shape[1]) and 
               grid[i, j] != 0):
            if display:
                plot[i, j] = 60 if len(path) == 0 else 30 if path[-1] == 'L' else 50
            i += o[0]
            j += o[1]
            forward += 1
        # One step back
        i -= o[0]
        j -= o[1]
        forward -= 1
        if forward:
            path.append(forward)
        # Check for next move
        path.append(find_nxt_move())
        forward = 0
    if display:
        display_grid(grid, "Original map")
        display_grid(plot, "Path")
    return path[:-1]

def display(program, inputs):
    out = run_program([x for x in program], inputs)[0]
    if out[-1] > ord('Z'):
        print(''.join(map(chr, out[:-1])))
        print('Hull damage =', out[-1])
    else:
        print(''.join(map(chr, out)))
    
    
def springcode_to_inputs(springcode, mode=0):
    cmd = [x for line in springcode if not line.startswith('#')
           for x in (list(map(ord, line)) + [10])]
    if mode == 0:
        return (cmd + [ord('W'), ord('A'), ord('L'), ord('K'), 10])[::-1]
    else:
        return (cmd + [ord('R'), ord('U'), ord('N'), 10])[::-1]

In [2]:
with open("inputs/day21.txt", 'r') as f:
    inputs = list(map(int, f.read().split(',')))

In [3]:
# Jump length is 4
### Assuming base rule = "always jump before a hole", some cases should be avoided:
# @.##.
#  -> previous view is @#.## = is it always safer to jump  immediately ?
# No, because od edge case @##.##.##. -> the only way is to jump at first position
# this leads to second rule

### Written with cases:
# If not A and D: jump (else = dead end)
# elIf not C and D: jump (else = depends ?, limited view)

### In Boolean
commands = """# Base command
NOT A J
AND D J
# Second test
NOT C T
AND D T 
OR T J"""

### Solving part 1
display([x for x in inputs], springcode_to_inputs(commands.splitlines()))

Input instructions:

Walking...


Hull damage = 19358416


In [4]:
# Jump length is 4
# For the second rule, we have a new edge case which can be detected with long view
# @##.#.#... -> it would be better to jump right before the hole
#
# We can also now easily detect new edge cases earlier like these:
#    1. @#.###.##. 
#       -> applying first rule would lead to dead end, instead we jump if B is a hole
#    2 @##.##.##. 
#       -> This is derived from the edge case in part 1, the current rule takes care of the
#          first jump, but not the second: @#.##.## -> we should jump immediately

### Cases
# If not A and D: jump (else = dead end)
# If not C and D and H
# If not B and not G and not I: jump
#   equiv to not (B or G or I)
# If not B and D and not E -> not (B or E)
# We can factorize:
#  - If NOT B
#  - AND D

### In Boolean
commands = """# New edge cases
# not G and not I
NOT G J
NOT I T
AND J T
# not E
NOT E J
OR J T
# add not B
NOT B J
AND T J
#Base command
NOT A T
OR T J
# Second test improved
NOT C T
AND H T
OR T J
#
# We can always distribute "AND D" to all commands
AND D J
"""


### Solving part 2
display([x for x in inputs], springcode_to_inputs(commands.splitlines(), mode=1))

Input instructions:

Running...


Hull damage = 1144641747
